In [ ]:
%load_ext autoreload
%autoreload 2

# This file contains tests draft for the Field. It is incomplete.
# It contains only tests which were used during development and which are useful.
import os
import sys
from datetime import date
import warnings
import shutil
from py.path import local
import numpy as np
import pandas as pd

sys.path.append('../..') # for running py-script
sys.path.append('../../..') # for running this notebook directly

warnings.filterwarnings('ignore')

from seismiqb import Field
from seismiqb.src.geometry import export

In [ ]:
# Defaults for run directly from this notebook
SEED = 10
SAVING_DIR = './'
DATESTAMP = date.today().strftime("%Y-%m-%d")
CUBE_SHAPE = (100, 100, 100)
DROP_EXTRA_FILES = True
GITHUB_MODE = False

# Preparation

**Storage structure:**
___



**field_test_files** (tests root directory with temporary files)

&emsp;├── **test_array.npy** (Test cube data, now it is a random noise)

&emsp;├── **test_cube.sgy** (Test cube data in the sgy format)

&emsp;├── **test_cube.meta** (Meta data of the test cube)
    
&emsp;└── **test_matrix** (Saved test matrix)


In [ ]:
# Storage preparation:
# The test_dir contains temporary files
test_dir = "field_tests_files"

if GITHUB_MODE:
    test_dir_path = SAVING_DIR.mkdir(test_dir)

else:
    # Clear and recreate workspace
    test_dir_path = os.path.join(SAVING_DIR, test_dir)

    try:
        shutil.rmtree(test_dir_path)
    except OSError as e:
        print(f"Can't delete the directory {test_dir_path} : {e.strerror}")

    if not os.path.exists(test_dir_path):
        os.makedirs(test_dir_path)

CUBE_PATH = os.path.join(test_dir_path, f'test_cube_{DATESTAMP}.sgy')

In [ ]:
%%time
# Create a fake cube
rng = np.random.default_rng(SEED)
data_array = rng.normal(0, 1000, CUBE_SHAPE).astype(np.float32)

with open(os.path.join(test_dir_path, f'test_array_{DATESTAMP}.npy'), 'wb') as outfile:
    np.save(outfile, data_array)

export.make_segy_from_array(array=data_array, path_segy=CUBE_PATH, zip_segy=False,
                            sample_rate=2., delay=50, pbar='t')

In [ ]:
%%time
# Init a field
field = Field(geometry=CUBE_PATH)

# Tests

In [ ]:
%%time
# Test dump_matrix (and dump_charisma under the hood), from_file and is_charisma_like

# Create and dump a random matrix
matrix = rng.integers(low=0, high=10, size=CUBE_SHAPE[:2])

path=os.path.join(test_dir_path, f'test_matrix_{DATESTAMP}')

field.dump_matrix(matrix=matrix, path=path)

# Check is_charisma_like
assert field.is_charisma_like(path), "A dumped file is not charisma-like."

# Open saved data
opened_matrix = field.from_file(path=path, return_points=False, name='opened_matrix')

assert np.array_equal(matrix, opened_matrix), "Saved and loaded matrices are not the same."

# Exit

In [ ]:
# Remove extra files
if DROP_EXTRA_FILES and not GITHUB_MODE:
    try:
        shutil.rmtree(test_dir_path)
    except OSError as e:
        print(f"Can't delete the directory {test_dir_path} : {e.strerror}")